In [8]:
import cv2
import numpy as np 


# Object Detection Using Contours

In [9]:
def nothing(x):
    pass

cv2.namedWindow('hancy')
cv2.createTrackbar("th","hancy",0,255,nothing)

cv2.createTrackbar("lb","hancy",0,255,nothing)
cv2.createTrackbar("lg","hancy",0,255,nothing)
cv2.createTrackbar("lr","hancy",0,255,nothing)

cv2.createTrackbar("hb","hancy",255,255,nothing)
cv2.createTrackbar("hg","hancy",255,255,nothing)
cv2.createTrackbar("hr","hancy",255,255,nothing)

cap = cv2.VideoCapture(0)
while cap.isOpened():
    r,frame = cap.read()
    if r == True:
        frame = cv2.flip(frame,1)
        frame = cv2.resize(frame,(400,400))
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        thr = cv2.getTrackbarPos('th','hancy')
        Lb = cv2.getTrackbarPos("lb","hancy")
        Lg = cv2.getTrackbarPos("lg","hancy")
        Lr = cv2.getTrackbarPos("lr","hancy")
    
        Ub = cv2.getTrackbarPos("hb","hancy")
        Ug = cv2.getTrackbarPos("hg","hancy")
        Ur = cv2.getTrackbarPos("hr","hancy")

        lower = np.array([Lb,Lg,Lr])
        upper = np.array([Ub,Ug,Ur])
        masks = cv2.inRange(hsv,lower,upper)
        res = cv2.bitwise_and(frame,frame,mask=masks)
        fr = cv2.bitwise_not(res)

        _,thi = cv2.threshold(masks,thr,255,cv2.THRESH_BINARY)
        cnt,hr = cv2.findContours(thi, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        cv2.drawContours(frame,cnt,-1,(255,0,0),2)
        cv2.imshow('thr',thi)
        cv2.imshow("res",res)
        cv2.imshow("mask",masks)
        cv2.imshow("cv",frame)
        if cv2.waitKey(25) & 0xff == ord("p"):
                break
    else:
        break

cap.release()  
cv2.destroyAllWindows()

# Image Background Using Histogram

In [7]:
img = cv2.imread("Images/thor.jpg")
img = cv2.resize(img,(600,500))
hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

img1 = cv2.imread("Images/cropthor.jpg")
hsv1 = cv2.cvtColor(img1,cv2.COLOR_BGR2HSV)

hist = cv2.calcHist([hsv1],[0,1],None,[180,256],[0,180,0,256])
mask = cv2.calcBackProject([hsv],[0,1],hist,[0,180,0,256],1)

# res = cv2.bitwise_or(img,img,mask = mask)
# this above commented one remove the thor but we want to remove background so let's make more filter
ker = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
mask = cv2.filter2D(mask,-1,ker)
_,thr = cv2.threshold(mask,230,255,cv2.THRESH_BINARY)
mask = cv2.merge((mask,mask,mask))

res = cv2.bitwise_or(img,mask)

cv2.imshow('res',res)
cv2.imshow("opencv",img)
cv2.waitKey()
cv2.destroyAllWindows()

# Hough Transformation Lines